# Asian OTC Options

This notebook demonstrates how to access and use the functinalities of **ETI Options** which are part of our **QPS** module within LSEG Financial Analytics SDK.

## Imports

In [1]:
from lseg_analytics.instruments.options import (
    value,
    OptionDefinition,
    OptionDefinitionInstrument,
    OptionPricingParameters,
    UnderlyingDefinition,
    ExerciseDefinition,
    ScheduleDefinition,
    AsianDefinition
)

from lseg_analytics.common import AdjustableDate
from datetime import date
import pandas as pd
import json as js

## Pricing an Asian OTC Option

In [2]:
definition = OptionDefinition(
    underlying = UnderlyingDefinition(
        code="AAPL.O",
        underlying_type="Equity"
    ),
    exercise = ExerciseDefinition(
        strike=240,
        exercise_style="European",
        schedule = ScheduleDefinition(
            end_date = AdjustableDate(
                date = date(2025, 8, 13)
            )
        )
    ),
    asian = AsianDefinition(
        asian_type="Price",
        average_type="Arithmetic",
        fixing_schedule= ScheduleDefinition(
            start_date=AdjustableDate(
                date=date(2025, 1, 18)
                ),
            end_date=AdjustableDate(
                date=date(2025, 3, 10)
                ),
            frequency="Monthly"
        ),
    ),
    option_type="Call"
)

pricing_parameters=OptionPricingParameters(
    valuation_date=date(2024, 12, 18)
)

response = value(
    definitions=[OptionDefinitionInstrument(definition=definition)],
    pricing_preferences=pricing_parameters
)

print(js.dumps(response.as_dict(), indent=4))


{
    "definitions": [
        {
            "definition": {
                "underlying": {
                    "underlyingType": "Equity",
                    "code": "AAPL.O"
                },
                "exercise": {
                    "strike": 240.0,
                    "schedule": {
                        "endDate": {
                            "dateType": "AdjustableDate",
                            "date": "2025-08-13"
                        }
                    },
                    "exerciseStyle": "European"
                },
                "optionType": "Call",
                "asian": {
                    "asianType": "Price",
                    "averageType": "Arithmetic",
                    "fixingSchedule": {
                        "startDate": {
                            "dateType": "AdjustableDate",
                            "date": "2025-01-18"
                        },
                        "endDate": {
                            "dateTyp

In [3]:
valuation = response.analytics[0].valuation

def flatten(data):
    rows = []

    for key, section in data.items():
        row = {"metric": key}

        # Start with raw values
        value = section.get("value")
        unit = section.get("unit")
        percent = section.get("percent")

        # Handle dealCurrency override
        dc = section.get("dealCurrency")
        if dc is not None:
            value = getattr(dc, "value", value)
            unit = getattr(dc, "currency", unit)

        # Round and assign
        if value is not None:
            row["value"] = round(value, 3)
        if unit is not None:
            row["unit"] = unit
        if percent is not None:
            row["percent"] = round(percent, 3)

        rows.append(row)

    return pd.DataFrame(rows).set_index("metric")


flatten(valuation)

,value,unit,percent
metric,,,
volatility,24.119,Percentage,NaN
marketValue,18.673,USD,NaN
totalMarketValue,18.673,USD,NaN
intrinsicValue,8.050,USD,NaN
timeValue,10.623,USD,NaN
premiumOverCash,10.623,USD,4.282
moneyness,103.354,Percentage,NaN
annualizedYield,0.065,Absolute,NaN
